## 03. Model Training - GreenSpace CNN

This notebook trains a simple multi-task CNN using the manifests produced in 02:
- Inputs: `data/processed/splits/{train,val,test}.csv`
- Backbone: EfficientNetB0 (ImageNet weights)
- Heads: 7×sigmoid (binaries), 3×softmax (shade), 5×softmax (score)



In [1]:
# Imports and paths
import os
import pandas as pd
import numpy as np
import tensorflow as tf
from pathlib import Path

train_csv = Path('../data/processed/splits/train.csv')
val_csv   = Path('../data/processed/splits/val.csv')
test_csv  = Path('../data/processed/splits/test.csv')

assert train_csv.exists() and val_csv.exists() and test_csv.exists(), 'Missing split manifests. Run 02 first.'

train_df = pd.read_csv(train_csv)
val_df   = pd.read_csv(val_csv)
test_df  = pd.read_csv(test_csv)

print('Loaded splits:', len(train_df), len(val_df), len(test_df))


Loaded splits: 29 10 10


In [2]:
# Build tf.data datasets from manifests
IMG_SIZE = (512, 512)
BATCH_SIZE = 8

# Identify label columns
binary_cols = [c for c in train_df.columns if c.endswith('_p') and not c.startswith(('shade_p_', 'score_p_'))]
shade_cols  = [c for c in train_df.columns if c.startswith('shade_p_')]
score_cols  = [c for c in train_df.columns if c.startswith('score_p_')]

print('Binary labels:', binary_cols)
print('Shade cols   :', shade_cols)
print('Score cols   :', score_cols)

# Map a row to (image, label dict)
def decode_image(path):
    img = tf.io.read_file(path)
    img = tf.io.decode_jpeg(img, channels=3)
    img = tf.cast(img, tf.float32) / 255.0
    return img

# Simple augmentation for train (same as 02)
def augment(img):
    k = tf.random.uniform((), minval=0, maxval=4, dtype=tf.int32)
    img = tf.image.rot90(img, k)
    delta = tf.random.uniform((), minval=-0.1, maxval=0.1)
    img = tf.clip_by_value(img + delta, 0.0, 1.0)
    return img

# Build a dataset from a DataFrame
def make_ds(df, augment_flag=False, shuffle=True):
    paths = df['image_path'].astype(str).tolist()
    y_bin = df[binary_cols].astype(np.float32).values
    y_shade = df[shade_cols].astype(np.float32).values
    y_score = df[score_cols].astype(np.float32).values

    ds_paths = tf.data.Dataset.from_tensor_slices(paths)
    ds_imgs = ds_paths.map(decode_image, num_parallel_calls=tf.data.AUTOTUNE)
    if augment_flag:
        ds_imgs = ds_imgs.map(augment, num_parallel_calls=tf.data.AUTOTUNE)

    ds_labels = tf.data.Dataset.from_tensor_slices({
        'bin_head': y_bin,
        'shade_head': y_shade,
        'score_head': y_score,
    })
    ds = tf.data.Dataset.zip((ds_imgs, ds_labels))
    if shuffle and len(paths) > 1:
        ds = ds.shuffle(buffer_size=len(paths), seed=123, reshuffle_each_iteration=True)
    ds = ds.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
    return ds

train_ds = make_ds(train_df, augment_flag=True, shuffle=True)
val_ds   = make_ds(val_df, augment_flag=False, shuffle=False)
test_ds  = make_ds(test_df, augment_flag=False, shuffle=False)

print('Datasets ready.')


Binary labels: ['sports_field_p', 'multipurpose_open_area_p', 'childrens_playground_p', 'water_feature_p', 'gardens_p', 'walking_paths_p', 'built_structures_p']
Shade cols   : ['shade_p_none', 'shade_p_some', 'shade_p_abundant']
Score cols   : ['score_p_1', 'score_p_2', 'score_p_3', 'score_p_4', 'score_p_5']


2025-09-10 09:42:19.421944: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M4
2025-09-10 09:42:19.422082: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2025-09-10 09:42:19.422096: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2025-09-10 09:42:19.422158: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2025-09-10 09:42:19.422179: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Datasets ready.


In [6]:
# Define a simple multi-head model (EfficientNetB0 backbone)
from tensorflow.keras import layers, models, applications, optimizers

INPUT_SHAPE = (512, 512, 3)
NUM_BIN = len(binary_cols)
NUM_SHADE = len(shade_cols)
NUM_SCORE = len(score_cols)

# Backbone (initialize without pretrained weights to avoid shape mismatch)
backbone = applications.EfficientNetB0(include_top=False, weights=None, input_shape=INPUT_SHAPE)
inputs = backbone.input
x = layers.GlobalAveragePooling2D()(backbone.output)

# Heads
bin_out = layers.Dense(NUM_BIN, activation='sigmoid', name='bin_head')(x)
shade_out = layers.Dense(NUM_SHADE, activation='softmax', name='shade_head')(x)
score_out = layers.Dense(NUM_SCORE, activation='softmax', name='score_head')(x)

model = models.Model(inputs=inputs, outputs=[bin_out, shade_out, score_out])

# Compile
losses = {
    'bin_head': 'binary_crossentropy',
    'shade_head': 'categorical_crossentropy',
    'score_head': 'categorical_crossentropy',
}
metrics = {
    'bin_head': ['accuracy'],
    'shade_head': ['accuracy'],
    'score_head': ['accuracy'],
}
model.compile(optimizer=optimizers.Adam(1e-3), loss=losses, metrics=metrics)

model.summary()



Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_3       │ (None, 512, 512,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ rescaling_6         │ (None, 512, 512,  │          0 │ input_layer_3[0]… │
│ (Rescaling)         │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ normalization_3     │ (None, 512, 512,  │          7 │ rescaling_6[0][0] │
│ (Normalization)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_conv_pad       │ (None, 513, 513,  │          0 │ normalization_3[… │
│ (ZeroPadding2D)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_conv (Conv2D)  │ (None, 256, 256,  │        864 │ stem_conv_pad[0]… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_bn             │ (None, 256, 256,  │        128 │ stem_conv[0][0]   │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_activation     │ (None, 256, 256,  │          0 │ stem_bn[0][0]     │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_dwconv      │ (None, 256, 256,  │        288 │ stem_activation[… │
│ (DepthwiseConv2D)   │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_bn          │ (None, 256, 256,  │        128 │ block1a_dwconv[0… │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_activation  │ (None, 256, 256,  │          0 │ block1a_bn[0][0]  │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_squeeze  │ (None, 32)        │          0 │ block1a_activati… │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_reshape  │ (None, 1, 1, 32)  │          0 │ block1a_se_squee… │
│ (Reshape)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_reduce   │ (None, 1, 1, 8)   │        264 │ block1a_se_resha… │
│ (Conv2D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_expand   │ (None, 1, 1, 32)  │        288 │ block1a_se_reduc… │
│ (Conv2D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_excite   │ (None, 256, 256,  │          0 │ block1a_activati… │
│ (Multiply)          │ 32)               │            │ block1a_se_expan… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_project_co… │ (None, 256, 256,  │        512 │ block1a_se_excit… │
│ (Conv2D)            │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_project_bn  │ (None, 256, 256,  │         64 │ block1a_project_

 Total params: 4,068,786 (15.52 MB)

 Trainable params: 4,026,763 (15.36 MB)

 Non-trainable params: 42,023 (164.16 KB)

In [7]:
# Train (warm-up then fine-tune)
EPOCHS_WARMUP = 5
EPOCHS_FINETUNE = 10

# Warm-up: freeze backbone, train heads
for layer in model.layers:
    if isinstance(layer, tf.keras.Model) or layer.name.startswith('efficientnet'):
        layer.trainable = False

callbacks = [
    tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True),
    tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2),
]

history_warmup = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=EPOCHS_WARMUP,
    callbacks=callbacks,
    verbose=1,
)

# Fine-tune: unfreeze top backbone blocks
for layer in model.layers:
    layer.trainable = True

model.compile(optimizer=tf.keras.optimizers.Adam(1e-4), loss=losses, metrics=metrics)

history_finetune = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=EPOCHS_FINETUNE,
    callbacks=callbacks,
    verbose=1,
)

print('Training complete.')


Epoch 1/5


2025-09-10 09:45:29.989744: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


4/4 ━━━━━━━━━━━━━━━━━━━━ 51s 5s/step - bin_head_accuracy: 0.0000e+00 - bin_head_loss: nan - loss: nan - score_head_accuracy: 0.0690 - score_head_loss: nan - shade_head_accuracy: 0.4483 - shade_head_loss: nan - val_bin_head_accuracy: 0.0000e+00 - val_bin_head_loss: nan - val_loss: nan - val_score_head_accuracy: 0.0000e+00 - val_score_head_loss: nan - val_shade_head_accuracy: 0.6000 - val_shade_head_loss: nan - learning_rate: 0.0010
Epoch 2/5
4/4 ━━━━━━━━━━━━━━━━━━━━ 10s 2s/step - bin_head_accuracy: 0.0000e+00 - bin_head_loss: nan - loss: nan - score_head_accuracy: 0.0000e+00 - score_head_loss: nan - shade_head_accuracy: 0.5517 - shade_head_loss: nan - val_bin_head_accuracy: 0.0000e+00 - val_bin_head_loss: nan - val_loss: nan - val_score_head_accuracy: 0.0000e+00 - val_score_head_loss: nan - val_shade_head_accuracy: 0.6000 - val_shade_head_loss: nan - learning_rate: 0.0010
Epoch 3/5
4/4 ━━━━━━━━━━━━━━━━━━━━ 9s 2s/step - bin_head_accuracy: 0.0000e+00 - bin_head_loss: nan - loss: nan - sco

In [9]:
# Evaluate on validation and calibrate thresholds for binary labels
import json
from sklearn.metrics import precision_recall_fscore_support, accuracy_score, mean_absolute_error

# 1) Predict on val
pred_bin, pred_shade, pred_score = model.predict(val_ds, verbose=0)

# 2) Build ground-truth arrays from val_df (hard labels)
# Binary ground truth order matches model outputs via binary_cols (strip trailing _p)
bin_names = [c[:-2] for c in binary_cols]
bin_names = [c for c in bin_names if c in val_df.columns]
y_bin_true = val_df[bin_names].astype(int).values

# Shade/Score ground truth (class indices)
y_shade_true = val_df['shade_class'].astype(int).values if 'shade_class' in val_df.columns else None
y_score_true = val_df['score_class'].astype(int).values if 'score_class' in val_df.columns else None

# 3) Calibration: pick threshold per binary label by maximizing F1 on val
thresholds = {}
metrics_bin = {}
ths = np.linspace(0.05, 0.95, 19)
for i, name in enumerate(bin_names):
    best_f1, best_t = -1.0, 0.5
    y_prob = pred_bin[:, i]
    y_true = y_bin_true[:, i]
    for t in ths:
        y_hat = (y_prob >= t).astype(int)
        p, r, f1, _ = precision_recall_fscore_support(y_true, y_hat, average='binary', zero_division=0)
        if f1 > best_f1:
            best_f1, best_t = f1, t
    thresholds[name] = float(best_t)
    # report at chosen threshold
    y_hat = (y_prob >= best_t).astype(int)
    p, r, f1, _ = precision_recall_fscore_support(y_true, y_hat, average='binary', zero_division=0)
    metrics_bin[name] = {'precision': float(p), 'recall': float(r), 'f1': float(f1)}

# 4) Shade/Score metrics on val (argmax)
metrics_val = {'binary': metrics_bin, 'shade': {}, 'score': {}}
if y_shade_true is not None:
    shade_pred_class = pred_shade.argmax(axis=1)
    acc_shade = accuracy_score(y_shade_true, shade_pred_class)
    metrics_val['shade']['accuracy'] = float(acc_shade)

if y_score_true is not None:
    score_pred_class = pred_score.argmax(axis=1) + 1  # classes 1..5
    acc_score = accuracy_score(y_score_true, score_pred_class)
    # expected score MAE
    classes = np.arange(1, pred_score.shape[1] + 1, dtype=np.float32)
    score_expected = (pred_score * classes).sum(axis=1)
    # Robust MAE: ignore rows with NaN in either y_true or prediction
    y_true = y_score_true.astype(np.float32)
    valid = (~np.isnan(score_expected)) & (~np.isnan(y_true))
    if valid.sum() > 0:
        mae_score = float(np.mean(np.abs(score_expected[valid] - y_true[valid])))
    else:
        mae_score = float('nan')
    metrics_val['score']['accuracy'] = float(acc_score)
    metrics_val['score']['mae_expected'] = float(mae_score)

# 5) Save thresholds
thr_path = Path('../data/processed/thresholds.json')
thr_path.parent.mkdir(parents=True, exist_ok=True)
with open(thr_path, 'w') as f:
    json.dump({'thresholds': thresholds}, f, indent=2)

print('Calibrated thresholds saved to', thr_path)
print('Binary metrics (val):')
for k, v in metrics_bin.items():
    print(f"  {k}: P={v['precision']:.2f} R={v['recall']:.2f} F1={v['f1']:.2f} @t={thresholds[k]:.2f}")
if 'accuracy' in metrics_val.get('shade', {}):
    print(f"Shade val accuracy: {metrics_val['shade']['accuracy']:.2f}")
if 'accuracy' in metrics_val.get('score', {}):
    print(f"Score val accuracy: {metrics_val['score']['accuracy']:.2f}; MAE(exp): {metrics_val['score']['mae_expected']:.2f}")


Calibrated thresholds saved to ../data/processed/thresholds.json
Binary metrics (val):
  sports_field: P=0.00 R=0.00 F1=0.00 @t=0.05
  multipurpose_open_area: P=0.00 R=0.00 F1=0.00 @t=0.05
  childrens_playground: P=0.00 R=0.00 F1=0.00 @t=0.05
  water_feature: P=0.00 R=0.00 F1=0.00 @t=0.05
  gardens: P=0.00 R=0.00 F1=0.00 @t=0.05
  walking_paths: P=0.00 R=0.00 F1=0.00 @t=0.05
  built_structures: P=0.00 R=0.00 F1=0.00 @t=0.05
Shade val accuracy: 0.60
Score val accuracy: 0.10; MAE(exp): nan


### 03. Model Training - Multi-task GreenSpace CNN

This notebook implements and trains the multitask CNN using TensorFlow/Keras:

## Multitask Architecture
- **Backbone**: EfficientNet/ResNet (ImageNet pretrained)
- **Task 1**: Structured rating (1-5 scale) → Regression head
- **Task 2**: Binary features (6 features) → Multi-binary classification
- **Task 3**: Shade level (3 classes) → Categorical classification

## Training Features
- Multitask loss weighting
- Data augmentation
- Learning rate scheduling  
- Early stopping
- Model checkpointing
